In [1]:
import pandas as pd


# df['raised_amount_usd'] = df['raised_amount_usd'].apply(lambda x: ("".join(x.split(","))).strip())
# df.loc[df.funding_total_usd == '-', 'raised_amount_usd'] = None
# df['raised_amount_usd'] = df['raised_amount_usd'].map(lambda x: float(x) if x is not None else x)


In [2]:
def split_backslash(x):
    try:
        return str(x).split("/")[1].strip()
    except:
        return x

In [3]:
raw_unicorns = pd.read_csv("scrapunicrons_2016/unicorns.csv",header=None)
list_unicorns = list(raw_unicorns[0])
list_unicorns_lower = [x.lower().replace(" ", "") for x in list_unicorns]

In [4]:
df = pd.read_csv("../raw_data_crunchbase/crunchbase-investments.csv")
# df = df[df['funded_year']==2013]
df['investor_permalink'] = df['investor_permalink'].apply(lambda x: split_backslash(x))
df = df.loc[df['investor_permalink']=='organization'] # change to person!!!!!!!

# from 20179 to 17617 for raised_amount_usd
df = df.loc[df["raised_amount_usd"].notnull()]
df['raised_amount_usd'] = df['raised_amount_usd'].apply(lambda x: "".join(x.split(",")).strip())
df.loc[df.raised_amount_usd == '-', 'raised_amount_usd'] = None

df['raised_amount_usd'] = df['raised_amount_usd'].map(lambda x: float(x) if x is not None else x)

# # all present for funding_round_type
df = df.loc[df["funded_quarter"].notnull()]

# df = df.loc[df["investor_country_code"].notnull()]
df = df.loc[df["company_market"].notnull()]

/Users/carlo_liquido/anaconda/envs/python2/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [5]:
df.shape

(76537, 24)

In [6]:
df.columns

Index([u'company_permalink', u'company_name', u'company_category_list',
       u'company_market', u'company_country_code', u'company_state_code',
       u'company_region', u'company_city', u'investor_permalink',
       u'investor_name', u'investor_category_list', u'investor_market',
       u'investor_country_code', u'investor_state_code', u'investor_region',
       u'investor_city', u'funding_round_permalink', u'funding_round_type',
       u'funding_round_code', u'funded_at', u'funded_month', u'funded_quarter',
       u'funded_year', u'raised_amount_usd'],
      dtype='object')

In [7]:
subset_df = df[["company_name", "company_country_code", "company_city", "company_market",
                "investor_name","investor_country_code","investor_city",
                "funded_at","funded_month","funded_year","funded_quarter","raised_amount_usd"]]

In [8]:
years_wanted = [2004,2005,2006,2007,2008,2009,2010,2011,2012,2013]

In [9]:
subset_df = subset_df.loc[subset_df['funded_year'].isin(years_wanted)]
# subset_df = subset_df[subset_df['funded_year']>=2004 and subset_df['funded_year']>=2013]

In [10]:
subset_df.head()

,company_name,company_country_code,company_city,company_market,investor_name,investor_country_code,investor_city,funded_at,funded_month,funded_year,funded_quarter,raised_amount_usd
13,AbilTo,USA,New York,Health and Wellness,.406 Ventures,USA,Boston,05/14/2013 00:00,2013-05,2013,2013-Q2,3000000
14,Ambient Devices,NaN,NaN,Mobile,.406 Ventures,USA,Boston,09/16/2008 00:00,2008-09,2008,2008-Q3,5000000
15,Urban Compass,USA,New York,Local,.406 Ventures,USA,Boston,12/17/2012 00:00,2012-12,2012,2012-Q4,8000000
16,MineralTree,USA,Cambridge,Analytics,.406 Ventures,USA,Boston,06/27/2012 00:00,2012-06,2012,2012-Q2,6300000
17,Connotate,USA,New Brunswick,Analytics,.406 Ventures,USA,Boston,04/12/2012 00:00,2012-04,2012,2012-Q2,7000000


In [11]:
# subset_df.to_csv("aggregated-investments-maptime.csv", index=False)

In [12]:
add_unicorns = subset_df.copy()

In [13]:
add_unicorns.shape

(66077, 12)

In [14]:
add_unicorns['company_name_lower'] = add_unicorns['company_name'].apply(lambda x: str(x).lower().replace(" ", ""))

In [15]:
add_unicorns.head()

,company_name,company_country_code,company_city,company_market,investor_name,investor_country_code,investor_city,funded_at,funded_month,funded_year,funded_quarter,raised_amount_usd,company_name_lower
13,AbilTo,USA,New York,Health and Wellness,.406 Ventures,USA,Boston,05/14/2013 00:00,2013-05,2013,2013-Q2,3000000,abilto
14,Ambient Devices,NaN,NaN,Mobile,.406 Ventures,USA,Boston,09/16/2008 00:00,2008-09,2008,2008-Q3,5000000,ambientdevices
15,Urban Compass,USA,New York,Local,.406 Ventures,USA,Boston,12/17/2012 00:00,2012-12,2012,2012-Q4,8000000,urbancompass
16,MineralTree,USA,Cambridge,Analytics,.406 Ventures,USA,Boston,06/27/2012 00:00,2012-06,2012,2012-Q2,6300000,mineraltree
17,Connotate,USA,New Brunswick,Analytics,.406 Ventures,USA,Boston,04/12/2012 00:00,2012-04,2012,2012-Q2,7000000,connotate


In [16]:
add_unicorns['unicorn_flag'] = add_unicorns['company_name_lower'].apply(lambda x: x in list_unicorns_lower)

In [17]:
# unicorns_df = add_unicorns[add_unicorns['unicorn_flag']==True]

In [18]:
top15 = ["Biotechnology", "Software", "Clean Technology", "Health Care", "E-Commerce", "Mobile", "Semiconductors", "Enterprise Software", "Advertising", "Hardware + Software", "Web Hosting", "Games", "Finance", "Curated Web", "Security"]

In [19]:
# unicorns_top25 = add_unicorns[add_unicorns["company_market"].isin(top25)]

In [20]:
# unicorns_top25.to_csv("top25-investments-unicorns.csv")

In [21]:
add_unicorns = add_unicorns[add_unicorns["company_market"].isin(top15)]
add_unicorns.head()

,company_name,company_country_code,company_city,company_market,investor_name,investor_country_code,investor_city,funded_at,funded_month,funded_year,funded_quarter,raised_amount_usd,company_name_lower,unicorn_flag
14,Ambient Devices,NaN,NaN,Mobile,.406 Ventures,USA,Boston,09/16/2008 00:00,2008-09,2008,2008-Q3,5000000,ambientdevices,False
18,Bit9,USA,Waltham,Security,.406 Ventures,USA,Boston,10/27/2007 00:00,2007-10,2007,2007-Q4,10000000,bit9,False
19,CloudHealth Technologies,USA,Boston,Software,.406 Ventures,USA,Boston,03/12/2013 00:00,2013-03,2013,2013-Q1,4500000,cloudhealthtechnologies,False
22,woodpellets.com,USA,Goffstown,Clean Technology,.406 Ventures,USA,Boston,08/31/2009 00:00,2009-08,2009,2009-Q3,11000000,woodpellets.com,False
23,woodpellets.com,USA,Goffstown,Clean Technology,.406 Ventures,USA,Boston,06/03/2008 00:00,2008-06,2008,2008-Q2,4000000,woodpellets.com,False


In [22]:
reshape_unicorns = (add_unicorns.groupby(['company_market','funded_quarter']).median().reset_index())[["company_market","funded_quarter","raised_amount_usd"]]

In [23]:
count_unicorns = (add_unicorns.groupby(['company_market','funded_quarter']).count().reset_index())[["company_market","funded_quarter","raised_amount_usd"]]

In [24]:
# count_unicorns[count_unicorns["company_market"]=="Biotechnology"]
# reshape_unicorns.head()
combined_market = reshape_unicorns.merge(count_unicorns, left_on=['company_market','funded_quarter'], right_on=['company_market','funded_quarter'])

In [25]:
combined_market=combined_market.rename(columns = {'raised_amount_usd_x':'funding','raised_amount_usd_y':'startup'})

In [26]:
combined_market.head()

,company_market,funded_quarter,funding,startup
0,Advertising,2004-Q1,250000,1
1,Advertising,2004-Q2,3000000,4
2,Advertising,2004-Q3,6250000,2
3,Advertising,2004-Q4,6000000,5
4,Advertising,2005-Q1,6000000,26


In [27]:
combined_top15 = combined_market[combined_market["company_market"].isin(top15)]

In [28]:
len(set(combined_top15.company_market))
combined_top15.shape

(586, 4)

In [29]:
all_unicorns_median = (add_unicorns.groupby(['funded_quarter']).median().reset_index())[["funded_quarter","raised_amount_usd"]]

In [30]:
all_unicorns_count = (add_unicorns.groupby(['funded_quarter']).count().reset_index())[["funded_quarter","raised_amount_usd"]]

In [39]:
all_unicorns = all_unicorns_median.merge(all_unicorns_count, left_on=['funded_quarter'], right_on=['funded_quarter'])
all_unicorns=all_unicorns.rename(columns = {'raised_amount_usd_x':'funding','raised_amount_usd_y':'startup'})
all_unicorns.insert(0, "company_market", "All")
all_unicorns.head()

,company_market,funded_quarter,funding,startup
0,All,2004-Q1,11000000,94
1,All,2004-Q2,7000000,83
2,All,2004-Q3,10400000,79
3,All,2004-Q4,10000000,80
4,All,2005-Q1,8500000,798


In [40]:
combined_top15_Final = all_unicorns.append(combined_top15)
combined_top15_Final.shape

(626, 4)

In [44]:
combined_top15_Final.to_csv("top15-pandas-quarter.csv", index=False)

In [43]:
combined_top15_Final[combined_top15_Final['company_market']=="Biotechnology"]

,company_market,funded_quarter,funding,startup
40,Biotechnology,2004-Q1,38000000,14
41,Biotechnology,2004-Q2,2327975,8
42,Biotechnology,2004-Q3,13500000,11
43,Biotechnology,2005-Q1,10000000,94
44,Biotechnology,2005-Q2,4206255,76
45,Biotechnology,2005-Q3,8990000,105
46,Biotechnology,2005-Q4,9540000,84
47,Biotechnology,2006-Q1,11800000,129
48,Biotechnology,2006-Q2,20000000,120
49,Biotechnology,2006-Q3,13960000,123


In [60]:
_unique = combined_top15_Final[['company_market','funded_quarter']]
len(_unique)

626

In [53]:
top15

['Biotechnology',
 'Software',
 'Clean Technology',
 'Health Care',
 'E-Commerce',
 'Mobile',
 'Semiconductors',
 'Enterprise Software',
 'Advertising',
 'Hardware + Software',
 'Web Hosting',
 'Games',
 'Finance',
 'Curated Web',
 'Security']

In [59]:
_months = set(list(combined_top15_Final['funded_quarter']))
_joined = [[x,y] for x in top15 for y in _months]
len(_joined)

600